In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def load_images(folder_path, label):
    """
    Loads grayscale images from a folder and returns them along with the label.
    """
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            # Resize to a fixed size (optional, here we use 128x128)
            img = cv2.resize(img, (128, 128))
            images.append(img)
            labels.append(label)
    return images, labels

def compute_features(image):
    """
    Extract several features from a given QR image:
      - Laplacian variance: Edge sharpness.
      - Mean intensity: Overall brightness.
      - Standard deviation: Contrast measure.
      - Dark pixel ratio: Proportion of dark pixels after thresholding.
    """
    # Laplacian variance for edge sharpness
    lap_var = cv2.Laplacian(image, cv2.CV_64F).var()

    # Mean and standard deviation of pixel intensities
    mean_intensity = np.mean(image)
    std_intensity = np.std(image)

    # Threshold image to binary (assumes QR codes are mostly black & white)
    _, thresh = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY)
    # Dark pixels (value 0) ratio
    dark_ratio = np.sum(thresh == 0) / thresh.size

    return lap_var, mean_intensity, std_intensity, dark_ratio

def analyze_dataset(original_folder, counterfeit_folder):
    # Load images from each folder
    orig_imgs, orig_labels = load_images(original_folder, 0)
    counterfeit_imgs, counterfeit_labels = load_images(counterfeit_folder, 1)

    # Combine datasets for later use if needed
    all_imgs = orig_imgs + counterfeit_imgs
    all_labels = orig_labels + counterfeit_labels

    # Extract features
    features = {"lap_var": [], "mean": [], "std": [], "dark_ratio": []}
    labels = []  # 0 for original, 1 for counterfeit
    for img, lab in zip(all_imgs, all_labels):
        f = compute_features(img)
        features["lap_var"].append(f[0])
        features["mean"].append(f[1])
        features["std"].append(f[2])
        features["dark_ratio"].append(f[3])
        labels.append(lab)

    features = {k: np.array(v) for k, v in features.items()}
    labels = np.array(labels)

    return features, labels, orig_imgs, counterfeit_imgs

def plot_feature_distributions(features, labels, feature_name, class_names=["Original", "Counterfeit"]):
    plt.figure(figsize=(8, 6))
    sns.histplot(features[feature_name][labels==0], color='blue', label=class_names[0], kde=True, stat="density", bins=20)
    sns.histplot(features[feature_name][labels==1], color='red', label=class_names[1], kde=True, stat="density", bins=20)
    plt.xlabel(feature_name)
    plt.title(f"Distribution of {feature_name}")
    plt.legend()
    plt.show()

# # Set your folder paths here
original_folder = "/content/drive/MyDrive/Datasets/QR_Classifier/First Print"      # e.g., "./data/original"
counterfeit_folder = "/content/drive/MyDrive/Datasets/QR_Classifier/Second Print"  # e.g., "./data/counterfeit"


# Analyze dataset and extract features
features, labels, orig_imgs, counterfeit_imgs = analyze_dataset(original_folder, counterfeit_folder)

# Print basic statistics
print("Feature Statistics:")
for key, vals in features.items():
    print(f"{key} (Original): mean={np.mean(vals[labels==0]):.2f}, std={np.std(vals[labels==0]):.2f}")
    print(f"{key} (Counterfeit): mean={np.mean(vals[labels==1]):.2f}, std={np.std(vals[labels==1]):.2f}")
    print("--------------------------------------------------")

# Plot feature distributions for comparison
plot_feature_distributions(features, labels, "lap_var")
plot_feature_distributions(features, labels, "mean")
plot_feature_distributions(features, labels, "std")
plot_feature_distributions(features, labels, "dark_ratio")

# Optionally, visualize a few sample images from each class
def show_samples(images, title, num=5):
    plt.figure(figsize=(15, 3))
    for i in range(num):
        plt.subplot(1, num, i+1)
        plt.imshow(images[i], cmap='gray')
        plt.title(title)
        plt.axis('off')
    plt.show()

show_samples(orig_imgs, "Original")
show_samples(counterfeit_imgs, "Counterfeit")


In [ ]:
import cv2
import numpy as np
import pandas as pd
from skimage.util import view_as_blocks
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os

# 1. QR Code Property Extraction Functions
# ========================================

def calculate_print_quality_features(image):
    """Extract print quality metrics"""
    # Edge sharpness
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    edge_sharpness = laplacian.var()

    # Contrast analysis
    contrast = np.max(image) - np.min(image)

    # Local contrast variance
    block_size = (32, 32)
    if image.shape[0] < 32 or image.shape[1] < 32:
        local_contrast = 0  # Avoid errors on small images
    else:
        windows = view_as_blocks(image, block_size)
        local_contrast = np.var([w.max() - w.min() for w in windows.reshape(-1, 32, 32)])

    return [edge_sharpness, contrast, local_contrast]

def analyze_structural_patterns(image):
    """Analyze QR code structural elements"""
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Finder pattern consistency (top-left corner)
    finder_region = binary[:21, :21]  # Assuming version 1 QR code
    finder_score = np.mean(finder_region == 255)

    # Alignment pattern detection
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = [cv2.boundingRect(c)[:2] for c in contours if 50 < cv2.contourArea(c) < 150]
    alignment_deviation = np.std(bounding_boxes) if bounding_boxes else 0

    return [finder_score, alignment_deviation]

def spectral_analysis(image):
    """Frequency domain features"""
    fft = np.fft.fft2(image)
    fshift = np.fft.fftshift(fft)
    magnitude = 20*np.log(np.abs(fshift) + 1e-9)  # Avoid log(0)

    # High frequency components
    high_freq = magnitude[30:70, 30:70].mean()
    # Low frequency components
    low_freq = magnitude[100:150, 100:150].mean() if magnitude.shape[0] > 150 else 0

    return [high_freq / (low_freq + 1e-9), np.std(magnitude)]

def ink_distribution_analysis(image):
    """Analyze ink distribution patterns"""
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    areas = [cv2.contourArea(c) for c in contours]
    eccentricities = []
    for c in contours:
        if len(c) >= 5:
            _, (major, minor), _ = cv2.fitEllipse(c)
            if major > 0:
                eccentricities.append(np.sqrt(1 - (minor / major) ** 2))

    return [np.var(areas) if areas else 0, np.mean(eccentricities) if eccentricities else 0]

# 2. Complete Feature Extraction Pipeline
# =======================================

def extract_qr_features(image):
    """Combine all quality metrics"""
    image = cv2.resize(image, (128, 128))  # Standardize image size
    features = []

    # Basic image properties
    features += [np.mean(image), np.std(image)]

    # Print quality metrics
    features += calculate_print_quality_features(image)

    # Structural analysis
    features += analyze_structural_patterns(image)

    # Spectral features
    features += spectral_analysis(image)

    # Ink distribution
    features += ink_distribution_analysis(image)

    return np.array(features)

# 3. Data Loading and Processing
# ==============================

def load_images_from_folder(folder, label):
    """Load images from a folder and assign labels"""
    images, labels = [], []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is not None:
            images.append(image)
            labels.append(label)
    return images, labels

def load_dataset(original_path, counterfeit_path):
    """Load original and counterfeit QR code images"""
    orig_images, orig_labels = load_images_from_folder(original_path, label=0)
    fake_images, fake_labels = load_images_from_folder(counterfeit_path, label=1)

    X = orig_images + fake_images
    y = orig_labels + fake_labels
    return X, np.array(y)

# 4. Model Training and Evaluation
# ================================

# Paths to datasets (Change these paths)
original_path = "/content/drive/MyDrive/Datasets/QR_Classifier/First Print"
counterfeit_path = "/content/drive/MyDrive/Datasets/QR_Classifier/Second Print"

# Load dataset
X, y = load_dataset(original_path, counterfeit_path)

# Extract features
print("Extracting features...")
X_features = np.array([extract_qr_features(img) for img in X])

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y, test_size=0.2, stratify=y, random_state=42
)

# Preprocessing
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train model
model = RandomForestClassifier(n_estimators=200,
                              class_weight='balanced',
                              random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# 5. Feature Importance Analysis
# ==============================
feature_names = [
    'mean_intensity', 'std_intensity',
    'edge_sharpness', 'global_contrast', 'local_contrast_var',
    'finder_pattern_score', 'alignment_deviation',
    'high_freq_ratio', 'spectral_std',
    'ink_area_variance', 'ink_eccentricity'
]

importances = model.feature_importances_
plt.figure(figsize=(10, 6))
plt.barh(feature_names, importances)
plt.title("Feature Importances")
plt.xlabel("Importance Score")
plt.show()

# 6. Deployment-ready Prediction
# ==============================
def authenticate_qrcode(image_path):
    """Complete authentication pipeline"""
    # Load and preprocess image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 128))

    # Extract features
    features = extract_qr_features(img)
    features = scaler.transform([features])

    # Predict
    proba = model.predict_proba(features)[0][1]
    return {
        'authentic': proba < 0.5,  # Assuming class 0 is authentic
        'confidence': 1 - abs(proba - 0.5) * 2
    }
